## Camera Calibration

You will need a chessboard pattern printed out and preferably taped to cardboard (so it doesn't bend).  

You may download a chessboard pattern here (https://raw.githubusercontent.com/opencv/opencv/4.x/doc/pattern.png), or you can create your own chessboard pattern using these steps:  https://docs.opencv.org/4.x/da/d0d/tutorial_camera_calibration_pattern.html

Additional info about the openCV calibration procedure may be found at https://docs.opencv.org/4.x/dc/dbb/tutorial_py_calibration.html and https://learnopencv.com/camera-calibration-using-opencv/ 

In [1]:
import sys, platform
from pathlib import Path

# ----------------------------------------------------------------------------------- 
# NOTE:  Update the path below to point to the directory where `ub_camera.py` is saved.
#        `Path` make the / operator work on Windows, macOS, Linux
# ----------------------------------------------------------------------------------- 
CODE_PATH = Path("/home/cmurray3/Projects/IE-482-582/spring2026/Projects/code")
SSL_PATH  = CODE_PATH / "ssl"

import ub_camera
import ub_utils
import cv2
import numpy as np

INFO: rospy is not installed and was not imported.  You may ignore this message.  Unless you are using ROS you do not need rospy.


In [ ]:
# Start a camera instance

# Initialize `CameraUSB` Class
# ----------------------------------------------------------------------------------- 
# NOTE:  You might need to update some of the parameters below:
#    - `port` is probably OK at 8001; change it if you get an error about a port already being in use.
#    - `apiPref` should be left as `None`
#    - `device` could also be the full URL to the "remote" camera stream
#    - `sslPath` should be the path on your machine where the `ssl` folder exists.  
# -----------------------------------------------------------------------------------
port      = ub_utils.findOpenPort(8001, options=range(8000,8040))
apiPref   = None   
device    = 0      # 'https://10.83.17.66:8002/stream.mjpg'  # 0      # '/dev/video0'  # '/dev/video4'
paramDict = {'res_rows':480, 'res_cols':640, 'fps_target':30, 'outputPort': port}


camera = ub_camera.CameraUSB(paramDict = paramDict, 
                             device = device, 
                             apiPref = apiPref,
                             sslPath = SSL_PATH,
                             showFPS=False)

camera.start(startStream=True, port=paramDict['outputPort'])

print(f'Visit https://localhost:{paramDict["outputPort"]}/stream.mjpg')
print("When you're done, be sure to stop the camera: camera.stop()")

## Additional Calibration Parameters:

We'll call the `addCalibrate()` function, which takes the following arguments:
- `res_rows` - Resolution rows (image height), in [pixels]. Defaults to 480.
- `res_cols` - Resolution columns (image width), in [pixels]. Defaults to 640.
- `secBetweenImages` - Number of seconds between images. Defaults to 3.
- `numImages` - Number of images required to perform calibration. Defaults to 30.
- `timeoutSec` - The program will terminate if it cannot find a checkerboard in this number of seconds. Defaults to 30.
- `pattern_size` - Pattern size of checkerboard (interior points).  Defaults to `tuple(6,8)`.
- `square_size` - Size of each square in the checkerboard, in [meters]. Defaults to 0.0254.
- `postFunction` - Name of a function that you'd like to run automatically when calibration is completed.  The function will take the following arguments:
    - `(success, res, mtx, dist, total_error, mean_error)`   

In [2]:
def printResults(success, res, mtx, dist, total_error, mean_error):
    try:
        if (success):
            '''
            mtx = [fx   0   cx
                    0  fy   cy
                    0   0    1]
            '''
            fx = mtx[0][0]
            fy = mtx[1][1]
            cx = mtx[0][2]
            cy = mtx[1][2]
            
            print('\n\n')
            print(f'Successful Calibration ({total_error=}, {mean_error=}).')
            print('------------ copy the part below ----------------')
            print(f'camera.intrinsics = {{ "{res}": {{"fx": {fx}, "fy": {fy}, "cx": {cx}, "cy": {cy}, "dist": {list(np.array(dist)[0])}  }} }}')
            print('camera.intrinsics = camera._getIntrinsics()')
            print('camera.intrinsics')
            print('-------------------------------------------------')
            
        else:
            print(f"Aruco Calibrate Failed: {success=}, {res=}, {mtx=}, {dist=}, {total_error=}, {mean_error=}.")

    except Exception as e:
        print(f"Error in printResults: {e}. {success=}, {mtx=}, {dist=}, {total_error=}, {mean_error=}")

In [3]:
# This will start the calibration procedure.
camera.addCalibrate(res_rows=paramDict['res_rows'], 
                    res_cols=paramDict['res_cols'], 
                    secBetweenImages=2,   # default is 3
                    numImages=30,         # 3 default is 30
                    timeoutSec=30, 
                    pattern_size=(6,8),   # <----- double-check this for your chessboard
                    square_size=0.0254,   # <----- double-check this for your chessboard
                    postFunction=printResults)   # The `printResults` function is defined above

LOGGER: Starting calibration default thread at 2 sec betw images
Camera matrix : 

[[844.48914657   0.         299.46002123]
 [  0.         848.10441642 236.49422747]
 [  0.           0.           1.        ]]
dist : 

[[ 0.12391035 -0.63306511 -0.00288657 -0.00367064  0.15891895]]
rvecs : 

[array([[0.37157505],
       [0.00710355],
       [1.37785789]]), array([[ 0.42503937],
       [-0.40697109],
       [ 1.48785357]]), array([[-0.09907691],
       [-0.28178599],
       [ 1.50758901]]), array([[-0.28024622],
       [ 0.21542674],
       [ 1.35944579]]), array([[-0.18446004],
       [-0.04658516],
       [ 1.00565635]]), array([[ 0.10984232],
       [-0.22097801],
       [-1.44405962]]), array([[ 0.25959075],
       [-0.26088834],
       [-1.44380311]]), array([[-0.32507116],
       [-0.21956893],
       [ 1.42099873]]), array([[-0.06488664],
       [-0.05681828],
       [ 1.36028155]]), array([[0.15262214],
       [0.33420628],
       [1.34447291]]), array([[-0.14915398],
       [ 0

In [6]:
# ****************************************************************************
# --------------- paste from the bottom of the output above ---------------- *
# ****************************************************************************
camera.intrinsics = { "640x480": {"fx": 613.9267755271052, "fy": 617.2876757419133, "cx": 326.06379688638367, "cy": 226.4726965669937, "dist": [-0.040671732389409375, 0.2205460570452358, -0.008313365917653356, 0.0025141234454979433, -0.32871689004906784]  } }
camera.intrinsics = camera._getIntrinsics()
camera.intrinsics

# You'll want to save the above info, for use in other projects that use your camera.

{'640x480': {'dist': array([-0.04067173,  0.22054606, -0.00831337,  0.00251412, -0.32871689]),
  'matrix': array([[613.92677553,   0.        , 326.06379689],
         [  0.        , 617.28767574, 226.47269657],
         [  0.        ,   0.        ,   1.        ]])}}

In [3]:
# Be sure to stop the camera when you're done:
camera.stop()